# Initial Exploration

In [130]:
# Imports
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import Pipeline

In [131]:
# Load the dataset
df = pd.read_csv("../data/raw/Agrofood_co2_emission.csv")

relevant_columns = ['Year', 'Area', 'total_emission', 'Savanna fires', 'Forest fires']
df = df[relevant_columns]

In [132]:
# Print data
print(df.head())
print("Time range: ", df['Year'].unique())

   Year         Area  total_emission  Savanna fires  Forest fires
0  1990  Afghanistan     2198.963539        14.7237        0.0557
1  1991  Afghanistan     2323.876629        14.7237        0.0557
2  1992  Afghanistan     2356.304229        14.7237        0.0557
3  1993  Afghanistan     2368.470529        14.7237        0.0557
4  1994  Afghanistan     2500.768729        14.7237        0.0557
Time range:  [1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003
 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017
 2018 2019 2020]


In [133]:
# Remove duplicates

# Check for exact duplicates across all columns
exact_duplicates = df.duplicated().sum()
print(f"Exact duplicates found: {exact_duplicates}")

# View duplicate rows
duplicate_rows = df[df.duplicated(keep=False)]
print(f"Total rows involved in duplication: {len(duplicate_rows)}")

# Check duplicates on specific key columns
key_duplicates = df.duplicated(subset=['Year', 'Area']).sum()
print(f"Duplicates based on Year and Area: {key_duplicates}")

# Remove exact duplicates (keep first occurrence)
df_cleaned = df.drop_duplicates(keep='first')
df_cleaned = df_cleaned.dropna(subset=relevant_columns)

cleaning_log = {
    'original_rows': len(df),
    'exact_duplicates_removed': len(df) - len(df_cleaned),
    'final_rows': len(df_cleaned)
}
print(f"Cleaning summary: {cleaning_log}")

Exact duplicates found: 0
Total rows involved in duplication: 0
Duplicates based on Year and Area: 0
Cleaning summary: {'original_rows': 6965, 'exact_duplicates_removed': 93, 'final_rows': 6872}


In [134]:
def identify_quality_issues(df):
    """Comprehensive data quality assessment"""
    issues = {}
    
    # Check for impossible values (domain-specific)
    if 'temperature' in df.columns:
        impossible_temps = df[(df['temperature'] < -50) | (df['temperature'] > 70)]
        issues['impossible_temperatures'] = len(impossible_temps)
    
    if 'energy_consumption' in df.columns:
        negative_energy = df[df['energy_consumption'] < 0]
        issues['negative_energy'] = len(negative_energy)
    
    # Check for future dates
    if 'timestamp' in df.columns:
        df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
        future_dates = df[df['timestamp'] > datetime.now()]
        issues['future_dates'] = len(future_dates)
    
    # Check for format inconsistencies
    for col in df.select_dtypes(include=['object']).columns:
        unique_patterns = df[col].astype(str).str.len().value_counts()
        if len(unique_patterns) > 10:  # Many different lengths suggest format issues
            issues[f'{col}_format_inconsistency'] = len(unique_patterns)
    
    return issues

# Run quality assessment
quality_report = identify_quality_issues(df_cleaned)
print("Data Quality Issues Found:")
for issue, count in quality_report.items():
    if count > 0:
        print(f"  {issue}: {count} records")

Data Quality Issues Found:
  Area_format_inconsistency: 31 records


In [135]:
# Split data

split_date = 2013
train_data = df_cleaned[df_cleaned['Year'] < split_date]
test_data = df_cleaned[df_cleaned['Year'] >= split_date]

X_train = train_data.drop(columns=train_data.columns.difference(['Savanna fires', 'Forest fires']))
y_train = train_data['total_emission']
X_test = test_data.drop(columns=test_data.columns.difference(['Savanna fires', 'Forest fires']))
y_test = test_data['total_emission']

print("Training data size: ", train_data.shape[0])
print("Test data size: ", test_data.shape[0])
print("Percent of total data (train): ", train_data.shape[0] / df_cleaned.shape[0] * 100)
print("Percent of total data (test): ", test_data.shape[0] / df_cleaned.shape[0] * 100)

Training data size:  5080
Test data size:  1792
Percent of total data (train):  73.9231664726426
Percent of total data (test):  26.07683352735739


In [136]:
# Cross validate

# Basic K-Fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Evaluate model performance
model = LinearRegression()
cv_scores = cross_val_score(model, X_train, y_train, cv=kfold, 
                           scoring='neg_mean_squared_error')

print("Cross-validation scores (negative MSE):", cv_scores)

Cross-validation scores (negative MSE): [-4.59605240e+10 -3.18091324e+10 -3.88250730e+10 -3.73666600e+10
 -3.45606866e+10]


In [ ]:
# Statistical preprocessing pipeline

# CORRECT: Learn all parameters from training data only
def create_preprocessing_pipeline(X_train, y_train):
    """Create preprocessing pipeline fitted on training data"""
    
    # 1. Missing value imputation
    imputer = SimpleImputer(strategy='median')
    X_train_imputed = imputer.fit_transform(X_train)
    
    # 2. Feature scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_imputed)
    
    # 3. Feature selection
    selector = SelectKBest(f_regression, k=10)
    X_train_selected = selector.fit_transform(X_train_scaled, y_train)
    
    # Return fitted preprocessors and transformed data
    preprocessors = {
        'imputer': imputer,
        'scaler': scaler,
        'selector': selector
    }
    
    return X_train_selected, preprocessors

def apply_preprocessing_pipeline(X_test, preprocessors):
    """Apply training preprocessing to test data"""
    
    # Apply in same order as training
    X_test_imputed = preprocessors['imputer'].transform(X_test)
    X_test_scaled = preprocessors['scaler'].transform(X_test_imputed)
    X_test_selected = preprocessors['selector'].transform(X_test_scaled)
    
    return X_test_selected

# Usage
X_train_processed, fitted_preprocessors = create_preprocessing_pipeline(X_train, y_train)
X_test_processed = apply_preprocessing_pipeline(X_test, fitted_preprocessors)

/Users/lukasbals/tomorrow-university/10_regression/sustainability-predictor/.venv/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:782: UserWarning: k=10 is greater than n_features=2. All the features will be returned.
  warnings.warn(


In [139]:
# Save data in the processed folder


y_train.to_csv("../data/processed/y_train.csv", index=False)
y_test.to_csv("../data/processed/y_test.csv", index=False)
np.savetxt("../data/processed/X_train.csv", X_train_processed, delimiter=",")
np.savetxt("../data/processed/X_test.csv", X_test_processed, delimiter=",")